# Explosion Example

In this notebook we will show how to animate colors and make objects disappear and appear at certain times.\
We do so by animating how a sphere grows as it turns red to finally explode into multiple smaller spheres.

In [1]:
#Make sure you have wilson installed
#(uncomment command if needed)
#!pip install -e ..

Once again, import all necessary packages.

In [2]:
import numpy as np
import wilson

Let's start with the exploding mother nuclei:

In [3]:
mother = wilson.Sphere("Mother", groups=["Mother"], description="I'm not feeling well...")

Another new thing are groups: We can organize objects into groups, presenting users a way to easier find them and toggle their visibility.

We want it to grow and then explode, i.e. disappear.

Growing is straight forward: We create a graph starting from a smaller radius to a bigger radius. \
For the explosion we simply set the radius to zero in a very short time.

In [4]:
mother.radius = np.array([
    [0.0, 1.0],
    [10.0,2.5],
    [10.01, 0.0]
])

Animating colors is a bit trickier. wilson uses color maps, i.e. it takes a scalar and looks the corresponding color up in a list.

Since we're lazy we want to use a predefined one. We can specify any color map from matplotlib or cmasher, or any other color map registered into matplotlib.\
Let's use the one called `hot` (shocking I know). We have to register it globally at the project root

In [5]:
explosion = wilson.Project("Explosion",
    author="wilson",
    description="An everyday explosion",
    speedRatio=1.0,
    colormap='hot')

Now we can create the color animation. Color maps are usually defined within a range of [0,1], but you are free to choose another range if you define your own.

Since 0.0 is dark and 1.0 light, the animation is really simple. We don't have to care about the color after the explosion as it won't be visible anyway.

In [6]:
mother.color = np.array([
    [0.0, 0.0],
    [10.0, 1.0]
])

Don't forget to add the sphere!

In [7]:
explosion.appendAnimatable(mother)

Next up are the fragments. Making them appear is very similar to making the first disappear earlier. \
Instead of setting a radius zero, we now just start with zero and increase it at the explosion.

Since all our fragments will have the same size, we can share this animation. If we just give the same array to all daughter nucleii, during serialization wilson assumes them to be unique and actually creates a copy each time.

To actually share the animation, we have to create the corresponding `Graph` object ourselves. (wilson checks if a graph was submitted earlier and if so produces a reference to the earlier one instead of creating a copy)

In [8]:
fragment_radii = wilson.Graph([
    [10.0, 0.0],
    [10.01, 0.3]
], "daughter radii")

We want them to fly into different directions. We do this by giving each daughter its own position animation with different ending points

In [9]:
for i in range(12):
    #create flight path
    endPosition = np.random.rand(3) * 10.0 - 5.0
    position = np.empty((2,4))
    position[0] = 10.0, 0.0, 0.0, 0.0
    position[1] = 15.0, *endPosition

    #add new daughter nuclei to project
    daughter = wilson.Sphere(f'Daughter{i}',
        description="Seeking freedom",
        radius=fragment_radii,
        position=position,
        color='grey blue')
    explosion.appendAnimatable(daughter)

With everything done, we now simply save the project. Once again we'll append it to the tutorial catalogue.

In [10]:
with wilson.Catalogue('tutorial.wlsn', 'a') as cat:
    cat.save('explosion', explosion)